Exploratory Data Analysis

*Author: Gavin Brumfield*

Dependencies

In [33]:
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sklearn.preprocessing import MinMaxScaler

Data

In [34]:
# Create a SQL connection to our SQLite database
conn = sqlite3.connect('challenge.db')

# Pull each table into a pandas DataFrame
access = pd.read_sql_query("SELECT * FROM access", conn)
five_hundred_cities = pd.read_sql_query("SELECT * FROM five_hundred_cities", conn)
variable_list = pd.read_sql_query("SELECT * FROM variable_list", conn)

Preprocessing

In [35]:
# Filter variable_list to only include variables with Category Code 'ACCESS'
# For ease of use when investigating the variables.
access_variables = variable_list[variable_list['Category Code'] == 'ACCESS']

In [36]:
# Applied a 50% weight to percent of population with low access, a 30% weight to children with low access, and a 20% weight to seniors to determine the overall rank of areas of highest impact.
ranked = access
ranked['Access_Score'] = ((ranked['PCT_LACCESS_POP15'] * 0.5) + (ranked['PCT_LACCESS_CHILD15'] * 0.3) + (ranked['PCT_LACCESS_SENIORS15'] * 0.2))
# Sorted by access score for initial investigation.
ranked = ranked.sort_values(['Access_Score'], ascending=False)

In [37]:
# Scaling access score for later combination with health score.
scaler = MinMaxScaler()
ranked['Access_Score'] = scaler.fit_transform(ranked[['Access_Score']])

# Write results to csv for Visualization in Tableau
ranked.to_csv('access_ranked.csv', index=False)

In [38]:
# Filter for only columns in five_hundred_cities that end in "CrudePrev"
# This is to filter out the columns that are not relevant to the analysis.
crude_prev = five_hundred_cities.filter(regex='CrudePrev')
# Append the columns from five_hundred_cities that are relevant to the analysis.
crude_prev = pd.concat([five_hundred_cities.iloc[:, 2:7], crude_prev], axis=1)

# Selecting only relevant prevalence columns, excluding doctor visits etc. and focusing on health issues.
# This is to filter out the columns that are not relevant to the analysis.
health_issues = crude_prev[['ARTHRITIS_CrudePrev',
                            'BINGE_CrudePrev',
                            'BPHIGH_CrudePrev',
                            'OBESITY_CrudePrev',
                            'DIABETES_CrudePrev',
                            'CHD_CrudePrev',
                            'KIDNEY_CrudePrev',
                            'PHLTH_CrudePrev',
                            'STROKE_CrudePrev',
                            'TEETHLOST_CrudePrev']]
# Append the columns from five_hundred_cities that are relevant to the analysis.
health_issues = pd.concat([five_hundred_cities.iloc[:, 2:7], health_issues], axis=1)

In [39]:
# Scaling data between range of 0 and 1 for scoring.
# Columns to scale
columns_to_scale = health_issues.iloc[:, 5:15]
# Create scaler
scaler = MinMaxScaler()
# Scale
scaled = scaler.fit_transform(columns_to_scale)
# Replace original columns with scaled
health_issues.iloc[:, 5:15] = scaled

In [40]:
# Weights for each health issue
diabetes_weight = 0.25  # Strongest link to diet, high cost associated with medications and treatment.
obesity_weight = 0.20  # Strong link to diet, high impact on overall health.
bphigh_weight = 0.15  # Strong link to diet, medium cost associated with medications and treatment.
chd_weight = 0.10  # Strong link to diet.
kidney_weight = 0.10  # Moderate link to diet, potential for serious complications 
stroke_weight = 0.08  # Strong link to diet, high impact on quality of life
plth_weight = 0.05  # Moderate link to diet, potential for serious complications
binge_weight = 0.04  # Moderate link to diet, mental health implications
arthritis_weight = 0.02  # Limited direct dietary impact, but diet can influence overall health and inflammation
teethlost_weight = 0.01  # Primarily a consequence of other health factors, but diet plays a role in oral health

# Calculate the overall health score for each city
health_issues['Health_Score'] = (health_issues['DIABETES_CrudePrev'] * diabetes_weight) + \
                                (health_issues['OBESITY_CrudePrev'] * obesity_weight) + \
                                (health_issues['BPHIGH_CrudePrev'] * bphigh_weight) + \
                                (health_issues['CHD_CrudePrev'] * chd_weight) + \
                                (health_issues['KIDNEY_CrudePrev'] * kidney_weight) + \
                                (health_issues['STROKE_CrudePrev'] * stroke_weight) + \
                                (health_issues['PHLTH_CrudePrev'] * plth_weight) + \
                                (health_issues['BINGE_CrudePrev'] * binge_weight) + \
                                (health_issues['ARTHRITIS_CrudePrev'] * arthritis_weight) + \
                                (health_issues['TEETHLOST_CrudePrev'] * teethlost_weight)

# Sorting by health score for initial investigation.
ranked_health = health_issues.sort_values(['Health_Score'], ascending=False)

# Write results to csv for Visualization in Tableau
ranked_health.to_csv('health_ranked.csv')

In [41]:
# Extract county FIPS code
ranked_health['PlaceFIPS'] = ranked_health['PlaceFIPS'].astype(str)
ranked_health['CountyFIPS'] = ranked_health['PlaceFIPS'].str[:5]

# Ensure FIPS in ranked is a string
ranked['FIPS'] = ranked['FIPS'].astype(str)

# Merge ranked_health and ranked on CountyFIPS
merged = pd.merge(ranked_health, ranked, left_on='CountyFIPS', right_on='FIPS', how='inner')

In [42]:
merged['Overall_Score'] = (merged['Access_Score'] * 0.6) + (merged['Health_Score'] * 0.4)

merged_final = merged.sort_values(['Overall_Score'], ascending=False).head(50)

# Write results to csv for Visualization in Tableau
merged_final.to_csv('merged_ranked.csv', index=False)
merged.to_csv('merged.csv', index=False)

Tyler, TX and Hopewell, VA counties should be our main focus as each occur most frequently in the top 50 ranked based on overall score.

In [43]:
tyler = merged_final[merged_final['County'] == 'Tyler']
hopewell = merged_final[merged_final['County'] == 'Hopewell']

t_pop = tyler['LACCESS_POP15'].sum()
t_child = tyler['LACCESS_CHILD15'].sum()
t_senior = tyler['LACCESS_SENIORS15'].sum()

h_pop = hopewell['LACCESS_POP15'].sum()
h_child = hopewell['LACCESS_CHILD15'].sum()
h_senior = hopewell['LACCESS_SENIORS15'].sum()

print("----------------------------------------------")
print("Total Impact")
print("----------------------------------------------")
print("Expected Reach:", t_pop + h_pop)
print("Expected Children Reach:", t_child + h_child)
print("Expected Senior Reach:", t_senior + h_senior)
print("----------------------------------------------")
print("Tyler County, TX")
print("----------------------------------------------")
print("Expected Reach:", t_pop)
print("Expected Children Reach:", t_child)
print("Expected Senior Reach:", t_senior)
print("----------------------------------------------")
print("Hopewell County, VA")
print("----------------------------------------------")
print("Expected Reach:", h_pop)
print("Expected Children Reach", h_child)
print("Expected Senior Reach", h_senior)



----------------------------------------------
Total Impact
----------------------------------------------
Expected Reach: 477479.97230820433
Expected Children Reach: 97339.34614307736
Expected Senior Reach: 77276.86267436568
----------------------------------------------
Tyler County, TX
----------------------------------------------
Expected Reach: 279896.5864341816
Expected Children Reach: 52905.21962143392
Expected Senior Reach: 44382.68639751912
----------------------------------------------
Hopewell County, VA
----------------------------------------------
Expected Reach: 197583.38587402273
Expected Children Reach 44434.12652164344
Expected Senior Reach 32894.176276846556
